# Installation of Dependencies


In [4]:
!pip -q install transformers faiss-cpu langchain_huggingface langchain_elasticsearch torch sentence-transformers chromadb langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.3/523.3 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s e

#Importing Libraries

In [12]:
import faiss
import numpy as np
from elasticsearch import Elasticsearch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BertTokenizer, BertForSequenceClassification
from sentence_transformers import SentenceTransformer
import chromadb
import os
from google.colab import userdata

#Load Models and Tokenizers

In [16]:
from huggingface_hub import login

# Your Hugging Face API token (You can find it in your Hugging Face account settings)
hf_api_token = userdata.get('HUGGING_FACE_API_KEY')


# Log in to Hugging Face
login(token=hf_api_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Load the pre-trained models and tokenizers for text generation, sentence embedding,
# and reranking.

# Load the SentenceTransformer model for encoding queries and documents
sentence_model = SentenceTransformer('all-MiniLM-L6-v2')  # Small, fast model for embeddings

In [15]:
from langchain_groq import ChatGroq

#Get your Groq AI API key from https://console.groq.com/
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

# Helper function to get the language model
def get_llm():
    """
    Returns the language model instance.

    This function initializes and returns a ChatGroq language model configured with the specified model name,
    temperature, maximum tokens, and other settings.

    Returns:
        ChatGroq: An instance of the ChatGroq language model.
    """
    llm = ChatGroq(
        model="mixtral-8x7b-32768",
        temperature=0,
        max_tokens=1024,
    )
    return llm

# Setup Elasticsearch for Textual Retrieval
# Elasticsearch is used here to perform text-based search on indexed documents.


In [ ]:
# Initialize Hugging Face Embeddings
embeddings = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


In [ ]:
from elasticsearch import Elasticsearch, ConnectionError
from langchain_elasticsearch import ElasticsearchStore

In [ ]:
from elasticsearch import Elasticsearch

# es = Elasticsearch(
#     ['https://2e8be0fdffac440795fcfbecf86079b4.us-central1.gcp.cloud.es.io'],
#     bearer_auth=('elastic', 'aPRfFBGj1FkOMgvm7XqkgAFJ'),
#     )
es = Elasticsearch(hosts=['https://2e8be0fdffac440795fcfbecf86079b4.us-central1.gcp.cloud.es.io'],  http_auth=('elastic', 'aPRfFBGj1FkOMgvm7XqkgAFJ'))

es.ping()

<ipython-input-19-ab2be17bd3cd>:7: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(hosts=['https://2e8be0fdffac440795fcfbecf86079b4.us-central1.gcp.cloud.es.io'],  http_auth=('elastic', 'aPRfFBGj1FkOMgvm7XqkgAFJ'))


True

In [ ]:
# Define the mapping for the index
index_body = {
    "mappings": {
        "properties": {
            "content": {"type": "text"}
        }
    }
}

# Create an index named 'movies'
index_name = 'movies'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")


Index 'movies' already exists.


# Load Reranking Model
# Load a pre-trained BERT model and tokenizer for reranking retrieved documents.


In [ ]:
# Load the tokenizer for the reranking model
rerank_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the BERT model for sequence classification (used for reranking)
rerank_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Get the ChatGroq language model
chat_groq_model = get_llm()

# Function for Advanced Query Transformation
# This function modifies and expands the input query for better retrieval results.


In [ ]:
def advanced_query_transformation(query):
    """
    Transforms the input query by adding synonyms, extensions, or modifying the structure
    for better search performance.

    Args:
        query (str): The original query.

    Returns:
        str: The transformed query with added synonyms or related terms.
    """
    # Example transformation: adding an OR clause with a related term
    expanded_query = query + " OR related_term"
    return expanded_query

# Function for Advanced Query Routing
# This function decides the retrieval method (textual or vector-based) based on the query type.


In [ ]:
def advanced_query_routing(query):
    """
    Determines the retrieval method based on the presence of specific keywords in the query.

    Args:
        query (str): The user's query.

    Returns:
        str: 'textual' if the query requires text-based retrieval, 'vector' otherwise.
    """
    if "specific_keyword" in query:
        return "textual"
    else:
        return "vector"

# Fusion Retrieval Function
# This function retrieves documents using both vector-based and textual retrieval methods.


In [ ]:
# Fusion Retrieval Function
def fusion_retrieval(query, top_k=5):
    """
    Retrieves the top_k most relevant documents using a combination of vector-based
    and textual retrieval methods.

    Args:
        query (str): The search query.
        top_k (int): The number of top documents to retrieve.

    Returns:
        list: A list of combined results from both vector and textual retrieval methods.
    """
    # Vector-based retrieval using sentence embeddings
    query_embedding = sentence_model.encode(query).tolist()
    vector_results = collection.query(query_embeddings=[query_embedding], n_results=min(top_k, len(documents)))

    # Textual retrieval using Elasticsearch
    es_body = {
        "size": top_k,  # Move size into body
        "query": {
            "match": {
                "content": query
            }
        }
    }
    es_results = es.search(index=index_name, body=es_body)
    es_documents = [hit["_source"]["content"] for hit in es_results['hits']['hits']]

    # Combine results from both retrieval methods
    combined_results = vector_results['documents'][0] + es_documents

    return combined_results


# Document Reranking Function
# This function reranks retrieved documents based on their relevance to the query.


In [ ]:
import torch.nn.functional as F

# Document Reranking Function
def rerank_documents(query, documents):
    """
    Reranks the retrieved documents based on their relevance to the query using a pre-trained
    BERT model.

    Args:
        query (str): The user's query.
        documents (list): A list of documents retrieved from the search.

    Returns:
        list: A list of reranked documents, sorted by relevance.
    """
    inputs = [rerank_tokenizer.encode_plus(query, doc, return_tensors='pt', truncation=True, padding=True) for doc in documents]

    # Use logits to get scores
    scores = []
    for input in inputs:
        outputs = rerank_model(**input)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)
        positive_class_probability = probabilities[:, 1].item()  # Assuming the second element represents the positive class
        scores.append(positive_class_probability)

    ranked_docs = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in ranked_docs]


# Context Selection and Compression
# This function selects and summarizes the context to be used in the final answer generation.


In [ ]:
# Load summarization model
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
def select_and_compress_context(documents):
    """
    Summarizes the content of the retrieved documents to create a compressed context.

    Args:
        documents (list): A list of documents to summarize.

    Returns:
        list: A list of summarized texts for each document.
    """
    summarized_context = []
    for doc in documents:
        input_length = len(doc.split())  # Calculate input length based on word count
        max_length = min(100, input_length)  # Set max_length to input_length if smaller than 100
        summary = summarizer(doc, max_length=max_length, min_length=5, do_sample=False)[0]['summary_text']
        summarized_context.append(summary)
    return summarized_context

# Answer Generation Function
# This function generates the final answer based on the provided context and query.


In [ ]:
# Answer Generation Function
def generate_answer(query, chunks, llm):
    """
    Generates an answer based on the input query and context chunks using a language model.

    Args:
        query (str): The user's query.
        chunks (list): A list of context chunks to inform the answer.
        llm (ChatGroq): An instance of the ChatGroq language model.

    Returns:
        str: The generated answer.
    """
    # Combine chunks into a single context string
    context = "\n\n".join(chunks)

    # Construct the prompt for the language model as a string
    prompt = f"""[INST]
Instruction: You're an expert in movie suggestions. Your task is to analyze carefully the context and come up with an exhaustive answer to the following question:
{query}

Here is the context to help you:

{context}

[/INST]"""

    # Invoke the language model with the prompt
    response = llm.invoke(prompt)  # Pass the prompt as a string directly

    # Since response is likely an AIMessage object, access the content directly
    generated_text = response.content

    return generated_text


# Full Advanced Retrieval-Augmented Generation (RAG) Pipeline
# This function orchestrates the entire process: query transformation, retrieval, reranking,
# context compression, and answer generation.


In [ ]:
def advanced_rag_pipeline(query):
    """
    The main pipeline function for the Advanced Retrieval-Augmented Generation (RAG) system.
    It processes the query, retrieves relevant documents, reranks them, selects and compresses
    the context, and finally generates an answer.

    Args:
        query (str): The user's input query.

    Returns:
        str: The final generated answer.
    """
    # Transform and route query
    transformed_query = advanced_query_transformation(query)
    retrieval_method = advanced_query_routing(transformed_query)

    # Retrieve documents using fusion retrieval
    retrieved_documents = fusion_retrieval(transformed_query)

    # Rerank documents based on relevance
    ranked_documents = rerank_documents(query, retrieved_documents)

    # Select and compress context for answer generation
    context = select_and_compress_context(ranked_documents)

    # Generate final answer based on the context
    final_answer = generate_answer(query, context, chat_groq_model)
    return final_answer

# Example Usage of the Advanced RAG Pipeline
# Demonstrate the use of the pipeline with an example query.


In [ ]:
import chromadb

# Initialize ChromaDB client and create collection
client = chromadb.Client()

# Define the collection name
collection_name = "movies"

try:
    # Attempt to create the collection in ChromaDB
    collection = client.create_collection(name=collection_name)
    print(f"Collection '{collection_name}' created successfully.")

    # Define the documents to be inserted into the collection
    documents = [
        {"id": "1", "content": "The Shawshank Redemption is a great movie to watch on a rainy day."},
        {"id": "2", "content": "Forrest Gump is an uplifting film perfect for a rainy afternoon."}
    ]

    # Extract the IDs and content for insertion
    ids = [doc["id"] for doc in documents]
    contents = [doc["content"] for doc in documents]

    # Insert documents into the collection
    collection.add(ids=ids, documents=contents)
    print("Documents inserted successfully.")

except Exception as e:
    print(f"Collection '{collection_name}' already exists. No need to create it again.")
    # Optionally, you could fetch the existing collection here
    collection = client.get_collection(name=collection_name)

except Exception as e:
    print(f"An error occurred: {e}")


Collection 'movies' created successfully.


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 13.6MiB/s]


Documents inserted successfully.


In [ ]:
# Example query
query = "What are some good movies to watch on a rainy day?"

# Run the query through the Advanced RAG Pipeline
answer = advanced_rag_pipeline(query)

# Output the generated answer
print(answer)

Thank you for considering me as an expert in movie suggestions! I'm here to help you find the perfect films to enjoy on a rainy day. Given your interest in thought-provoking and uplifting movies, I have a few recommendations that might suit your taste.

1. The Shawshank Redemption (1994): This movie is a compelling and emotional story about friendship, hope, and redemption. It's a great choice for a rainy day, as it invites introspection and offers a satisfying resolution.
2. Forrest Gump (1994): As you mentioned, this is an uplifting film that explores the extraordinary life of a simple man. With its heartwarming story and stunning visuals, Forrest Gump is sure to brighten your rainy day.
3. The Green Mile (1999): Another thought-provoking film by the same director as The Shawshank Redemption, this movie is a supernatural drama set on death row. It's a touching and emotional story that will leave you reflecting on the power of empathy and the human spirit.
4. The Pursuit of Happyness 